# Query Reward Unit Test
Performing verification of the query rewards mechanism by comparing the event data to the results of cadCAD model.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal
from decimal import *
import model.parts.utils as utils
from model.sys_params import *
df = pd.read_pickle(r'experiment.p')

Context(prec=6, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])
Context(prec=6, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])
Context(prec=6, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])
stakeDelegateds: 611 events
stakeDelegatedLockeds: 380 events
stakeDelegatedWithdrawns: 331 events
allocationCloseds: 53 events
allocationCollecteds: 18 events
stakeDepositeds: 33 events
rewardsAssigneds: 53 events
delegationParametersUpdateds: 8 events
TOTAL NUMBER OF EVENTS: 1576
You should set SIMULATION_TIME_STEPS in config.py to a minimum of 1576 to capture all events.



Configure environment

In [2]:
df.reset_index(inplace = True)
pd.set_option('display.max_rows', None)

In [3]:
# print(query_fee_events)
query_fee_events_dict = query_fee_events
print("EXPECTED TRUE QUERY REWARDS FROM STAKE DEPOSITED EVENTS:")
for timestep, query_fee_events_list in query_fee_events_dict.items():
    if query_fee_events_list is None:
        query_fee_amt = 0
    else:
        query_fee_amt = sum([e['tokens'] for e in query_fee_events_list])
    print(f"{timestep}, {query_fee_amt}")
    


EXPECTED TRUE QUERY REWARDS FROM STAKE DEPOSITED EVENTS:
446, 0.350987412670748874
447, 0.619137696904111205
474, 137.509689357001388186
723, 26.735411802895408127
724, 177.886072492066675018
798, 214.529777397159839053
838, 18.276136055873106202
839, 18.276136055873106202
840, 18.276136055873106202
841, 18.276136055873106202
842, 18.276136055873106202
843, 18.276136055873106202
844, 18.276136055873106202
845, 18.276136055873106202
846, 18.276136055873106202
1111, 7.259243672384625343
1123, 1.200768983572724847
1138, 1.200768983572724847


In [4]:
print("MODELED RESULTS")
for timestep, query_fee_events_list in query_fee_events_dict.items():    
    event = query_fee_events_list[0]
    new_query_fee_amt = df.iloc[timestep-1].indexers[event['indexer']].cumulative_query_revenue
    old_query_fee_amt = df.iloc[timestep-2].indexers[event['indexer']].cumulative_query_revenue
    query_fee_amt = new_query_fee_amt - old_query_fee_amt
    print(f"{timestep}, {query_fee_amt=}")


MODELED RESULTS
446, query_fee_amt=Decimal('0.350987412670748874')
447, query_fee_amt=Decimal('0.619137696904111205')
474, query_fee_amt=Decimal('137.509689357001388186')
723, query_fee_amt=Decimal('26.735411802895408127')
724, query_fee_amt=Decimal('177.886072492066675018')
798, query_fee_amt=Decimal('214.529777397159839053')
838, query_fee_amt=Decimal('18.276136055873106202')
839, query_fee_amt=Decimal('18.276136055873106202')
840, query_fee_amt=Decimal('18.276136055873106202')
841, query_fee_amt=Decimal('18.276136055873106202')
842, query_fee_amt=Decimal('18.276136055873106202')
843, query_fee_amt=Decimal('18.276136055873106202')
844, query_fee_amt=Decimal('18.276136055873106202')
845, query_fee_amt=Decimal('18.276136055873106202')
846, query_fee_amt=Decimal('18.276136055873106202')
1111, query_fee_amt=Decimal('7.259243672384625343')
1123, query_fee_amt=Decimal('1.200768983572724847')
1138, query_fee_amt=Decimal('1.200768983572724847')


In [5]:
print("UNITTEST RESULTS")
cntExact = 0
cntReallyClose = 0
cntClose = 0
cnt = 0
cntWrong = 0
for timestep, query_fee_events_list in query_fee_events_dict.items():
    if query_fee_events_list is None:
        event_query_fee_amt = 0
    else:
        event_query_fee_amt = sum([e['tokens'] for e in query_fee_events_list])
    event = query_fee_events_list[0]
    new_query_fee_amt = df.iloc[timestep-1].indexers[event['indexer']].cumulative_query_revenue
    old_query_fee_amt = df.iloc[timestep-2].indexers[event['indexer']].cumulative_query_revenue
    model_query_fee_amt = new_query_fee_amt - old_query_fee_amt
    print(f"Event: Timestep={timestep-1}, Query Reward Tokens={event_query_fee_amt}")
    print(f"Model: Timestep={timestep-1}, Query Reward Tokens={model_query_fee_amt}")
    # print("Query Rewards Equal?", event_query_fee_amt == model_query_fee_amt)
    # print()

    try:
        ratio =  abs(event_query_fee_amt / model_query_fee_amt)
    except:
        ratio = np.inf
    exact = ratio == 1.0
    close = 0.99 <= ratio <= 1.01
    reallyClose = 0.99999 <= ratio <= 1.00001

    if exact:            
        cntExact += 1
    elif reallyClose:
        cntReallyClose += 1
    elif close:
        cntClose += 1
    else:
        cntWrong += 1
    cnt += 1
    print("Tokens Equal?", exact)
    print("Tokens Within 0.001%?", reallyClose)
    print("Tokens Within 1%?", close)
    print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Wrong: {cntWrong}, Total Number: {cnt}")   
    print()
print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Wrong: {cntWrong}, Total Number: {cnt}")   
    
    


UNITTEST RESULTS
Event: Timestep=445, Query Reward Tokens=0.350987412670748874
Model: Timestep=445, Query Reward Tokens=0.350987412670748874
Tokens Equal? True
Tokens Within 0.001%? True
Tokens Within 1%? True
Exact: 1, ReallyClose: 0, Close: 0, Wrong: 0, Total Number: 1

Event: Timestep=446, Query Reward Tokens=0.619137696904111205
Model: Timestep=446, Query Reward Tokens=0.619137696904111205
Tokens Equal? True
Tokens Within 0.001%? True
Tokens Within 1%? True
Exact: 2, ReallyClose: 0, Close: 0, Wrong: 0, Total Number: 2

Event: Timestep=473, Query Reward Tokens=137.509689357001388186
Model: Timestep=473, Query Reward Tokens=137.509689357001388186
Tokens Equal? True
Tokens Within 0.001%? True
Tokens Within 1%? True
Exact: 3, ReallyClose: 0, Close: 0, Wrong: 0, Total Number: 3

Event: Timestep=722, Query Reward Tokens=26.735411802895408127
Model: Timestep=722, Query Reward Tokens=26.735411802895408127
Tokens Equal? True
Tokens Within 0.001%? True
Tokens Within 1%? True
Exact: 4, Really

# Conclusion
The query reward token quantities are exactly equal.